# Day 14
--- Day 14: Extended Polymerization ---
The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has polymerization equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.

The submarine manual contains instructions for finding the optimal polymer formula; specifically, it offers a polymer template and a list of pair insertion rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.

For example:

NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C
The first line is the polymer template - this is the starting point of the process.

The following section defines the pair insertion rules. A rule like AB -> C means that when elements A and B are immediately adjacent, element C should be inserted between them. These insertions all happen simultaneously.

So, starting with the polymer template NNCB, the first step simultaneously considers all three pairs:

The first pair (NN) matches the rule NN -> C, so element C is inserted between the first N and the second N.
The second pair (NC) matches the rule NC -> B, so element B is inserted between the N and the C.
The third pair (CB) matches the rule CB -> H, so element H is inserted between the C and the B.
Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.

After the first step of this process, the polymer becomes NCNBCHB.

Here are the results of a few steps using the above rules:

Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, B occurs 1749 times, C occurs 298 times, H occurs 161 times, and N occurs 865 times; taking the quantity of the most common element (B, 1749) and subtracting the quantity of the least common element (H, 161) produces 1749 - 161 = 1588.

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

In [ ]:
public (string startingPoint, Dictionary<string, string> rules) Parse(string txt)
{
    var lines = txt.Split('\n', StringSplitOptions.RemoveEmptyEntries);
    var startingPoint = lines[0].Trim();
    var rules = new Dictionary<string, string>();

    for (int i = 1; i < lines.Length; i++)
    {
        var line = lines[i].Trim();
        var parts = line.Split(' ');

        if (parts.Length > 1)
            rules.Add(parts[0], parts[2]);
    }

    return (startingPoint, rules);
}

public string PerformStep(string startingPoint, Dictionary<string, string> rules)
{
    List<char> inserted = new List<char>();
    
    for (int i = 0; i < startingPoint.Length - 1; i++)
    {
        var current = (startingPoint[i].ToString() + startingPoint[i + 1].ToString());
        inserted.Add(rules.ContainsKey(current) ? rules[current][0] : '.');
    }

    // merge startingpoint and inserted
    var sb = new StringBuilder();
    sb.Append(startingPoint[0]);

    for (int i = 0; i < inserted.Count; i++)
    {
        if (inserted[i] != '.')
        {
            sb.Append(inserted[i]);
            sb.Append(startingPoint[i + 1]);
        }
        else
        {
            sb.Append(startingPoint[i]);
        }
    }

    return sb.ToString();
}

# Example

In [ ]:
var example = @"NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C";



In [ ]:
public long Result(string polymer)
{
    var counts = polymer.GroupBy(c => c).ToDictionary(g => g.Key, g => g.Count());
    return counts.Values.Max() - counts.Values.Min();
}

var (startingPoint, rules) = Parse(example);

for (int i = 0; i < 10; i++)
{
    startingPoint = PerformStep(startingPoint, rules);
    Console.WriteLine($"Step {i + 1}...");
}

Console.WriteLine($"Result: {Result(startingPoint)}\n");

(startingPoint, rules) = Parse(System.IO.File.ReadAllText("input.txt"));

for (int i = 0; i < 10; i++)
{
    startingPoint = PerformStep(startingPoint, rules);
    Console.WriteLine($"Step {i + 1}...");
}

Console.WriteLine($"Result: {Result(startingPoint)}");


Step 1...
Step 2...
Step 3...
Step 4...
Step 5...
Step 6...
Step 7...
Step 8...
Step 9...
Step 10...
Result: 1588

Step 1...
Step 2...
Step 3...
Step 4...
Step 5...
Step 6...
Step 7...
Step 8...
Step 9...
Step 10...
Result: 3118


In [ ]:

// Part 2 - 40 steps

long Result2(string txt, int steps)
{
    var (temp, rules) = Parse(txt);

    var pairCounts = new Dictionary<(char, char), long>();

    for (int i = 0; i < temp.Length - 1; i++)
        AddOrIncrease(pairCounts, (temp[i], temp[i + 1]), 1);

    for (int i = 0; i < steps; i++)
    {
        var keys = pairCounts.Keys.ToList();
        var newPairCounts = new Dictionary<(char, char), long>(pairCounts);
        
        foreach (var k in keys)
        {
            var key = k.Item1.ToString() + k.Item2.ToString();
            var count = pairCounts[k];
            
            if (rules.TryGetValue(key, out var value))
            {
                // Console.WriteLine($"Applying rule: {k.Item1} {k.Item2} -> {value}. Count: {count}");
                AddOrIncrease(newPairCounts, (k.Item1, value[0]), count);
                AddOrIncrease(newPairCounts, (value[0], k.Item2), count);
                RemoveOrDecrease(newPairCounts, k, count);
            }
        }

        pairCounts = newPairCounts;
    }

    // dump pairCounts
    // foreach (var k in pairCounts.Keys)
    //     Console.WriteLine($"{k.Item1} {k.Item2} {pairCounts[k]}");

    var count1 = pairCounts.GroupBy(k => k.Key.Item1).ToDictionary(g => g.Key, g => g.Sum(k => k.Value));
    var count2 = pairCounts.GroupBy(k => k.Key.Item2).ToDictionary(g => g.Key, g => g.Sum(k => k.Value));

    var elementCounts = new Dictionary<char, long>();
    
    foreach (var k in count1.Keys)
        elementCounts[k] = count1[k];

    AddOrIncrease(elementCounts, temp.Last());



    // dump elementCounts
    // foreach (var k in elementCounts.Keys)
    //     Console.WriteLine($"Count of '{k}': {elementCounts[k]}");

    var maxCount = elementCounts.Values.Max();
    var minCount = elementCounts.Values.Min();

    return maxCount - minCount;
}

void AddOrIncrease<T>(Dictionary<T, long> dict, T key, long amount = 1)
{
    if (dict.ContainsKey(key))
        dict[key] += amount;
    else
        dict.Add(key, amount);
}

void RemoveOrDecrease<T>(Dictionary<T, long> dict, T key, long amount = 1)
{
    if (dict.ContainsKey(key))
    {
        dict[key] -= amount;
        
        if (dict[key] == 0)
            dict.Remove(key);
    }
}

var steps = 40;
// Console.WriteLine($"Example; {steps} step(s): Result = {Result2(example, steps)}\n");
Console.WriteLine($"Input; {steps} step(s): Result = {Result2(System.IO.File.ReadAllText("input.txt"), steps)}");


Input; 40 step(s): Result = 4332887448171
